<a href="https://colab.research.google.com/github/kaisacht/Project/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Khai báo các thư viện cần dùng
from __future__ import print_function 
from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
#Khởi tạo ngẫu nhiên nhưng mà cố định
np.random.seed(813306)

In [ ]:
#hàm tải file lên
from google.colab import files
uploaded = files.upload()

In [ ]:
#tách dữ liệu thành 2 phần x và nhãn y tương ứng
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

In [ ]:
#tính toán ra tập x,y train và tập test
x_train, y_train = readucr('dataset_3class_2day_TRAIN.txt')
x_test, y_test = readucr('dataset_3class_2day_TEST.txt')

In [ ]:

#in thử tập dữ liệu và nhãn
print(x_test,y_test)

In [ ]:
#số lần trainning là 2000
nb_epochs = 2000

In [ ]:
#tính toán tập nhãn và dữu liệu cho tập train và test
x_train, y_train = readucr('dataset_3class_2day_TRAIN.txt')
x_test, y_test = readucr('dataset_3class_2day_TEST.txt')
#tính số lượng class
nb_classes = len(np.unique(y_test))
#tính tham số batch
batch_size = min(x_train.shape[0]/10, 16)
#đánh dấu lại nhãn phải băt đầu từ 0
y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
#?
Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_test = keras.utils.to_categorical(y_test, nb_classes)
#tính trung bình cộng của tập x
x_train_mean = x_train.mean()
#tính phương sai của tập x
x_train_std = x_train.std()
#chuẩn hóa lại tập giá trị của tập x
x_train = (x_train - x_train_mean)/(x_train_std)
#chuẩn hóa lại tập giá trị của tập y
x_test = (x_test - x_train_mean)/(x_train_std)
#chắc chắn ma trận của tập x có dạng (x.hape,1,1)
x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))
#?
x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
conv1 = keras.layers.Conv2D(128, 8, 1, padding='same')(x)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
conv2 = keras.layers.Conv2D(256, 5, 1, padding='same')(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)
    
#    drop_out = Dropout(0.2)(conv2)
conv3 = keras.layers.Conv2D(128, 3, 1, padding='same')(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)
    
full = keras.layers.GlobalAveragePooling2D()(conv3)
out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
#xây dựng model
model = keras.models.Model(inputs=x, outputs=out)
     
optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
#trainning
hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.

In [ ]:
#in ra lịch sử trainning
log = pd.DataFrame(hist.history)
print(log)

In [ ]:
#in ra lần train có hàm loss nhỏ nhất
log_np = np.array(log)
log_np
min_index = (np.argmin(log_np, axis=0))
print(log_np[min_index[0]])